In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataset_tools import *

In [2]:

# l = [i for i in open('/media/hpc2_storage/nvaulin/test_ties32/list.txt') if i.find('PTZ') < 0]
# f = open('/media/hpc2_storage/nvaulin/test_ties32/list.txt','w')
# for i in l:
#     f.write(i)
    
# f.close()

In [3]:
def get_motion_ties(img,_mask,t_size):
    mask = np.zeros_like(_mask,dtype=np.float32)
    mask[_mask > 240] = 1
    c = np.cumsum(mask,0)
    c = np.cumsum(c,1)
    mask = (c[t_size:,t_size:] + c[:-t_size,:-t_size] - c[:-t_size,t_size:] - c[t_size:,:-t_size])/(t_size**2)
    h = w = t_size//2
    cols = mask.shape[1]//w
    rows = mask.shape[0]//h
    mask = mask[:h*rows,:w*cols]
    x_inx = np.arange(mask.shape[1])[None,:]+mask*0
    y_inx = np.arange(mask.shape[0])[:,None]+mask*0
    mask = np.transpose(mask.reshape((rows,h,cols,w)),(0,2,1,3)).reshape(cols*rows,h*w)
    y_inx = np.transpose(y_inx.reshape((rows,h,cols,w)),(0,2,1,3)).reshape(cols*rows,h*w).astype(np.int32)
    x_inx = np.transpose(x_inx.reshape((rows,h,cols,w)),(0,2,1,3)).reshape(cols*rows,h*w).astype(np.int32)
    res = []
    for i in np.arange(len(mask))[mask.max(1) > 0.95]:
        inx = mask[i].argmax()
        x = x_inx[i][inx]
        y = y_inx[i][inx]
        res.append(img[y:y+t_size,x:x+t_size])
    return np.array(res)

def get_motion_batch(imgs,masks,t_size):
    res = np.empty((0,t_size,t_size,3),dtype=np.uint8)
    for i in range(len(imgs)):
        tmp = get_motion_ties(imgs[i],masks[i],t_size)
        if(len(tmp) > 0):
            res = np.concatenate((res,tmp),axis=0)
    return res

def process_batch(out_dir,imgs,masks,w,h,min_r,min_roi):
    cols = imgs.shape[2]//w
    rows = imgs.shape[1]//h
    imgs = imgs[:,:h*rows,:w*cols,:]
    masks = masks[:,:h*rows,:w*cols]
    imgs = np.transpose(imgs.reshape((-1,rows,h,cols,w,3)),(1,3,0,2,4,5)).reshape(cols*rows,len(imgs),h,w,3)
    masks = np.transpose(masks.reshape((-1,rows,h,cols,w)),(1,3,0,2,4)).reshape(cols*rows,len(masks),h,w)
    motion = np.zeros_like(masks)
    motion[masks > 240] = 1
    motion = motion.mean((2,3))

    roi = np.zeros_like(masks)
    roi[(masks > 240) | (masks < 10)] = 1
    roi = roi.mean((2,3))
    iter = 0
    for i in range(len(masks)):
        if(motion[i].mean() >= min_r and roi[i].mean() >= min_roi):
            cv2.imwrite(out_dir+'/%d_input.jpg'%(iter),ties_to_image(imgs[i]))
            cv2.imwrite(out_dir+'/%d_mask.png'%(iter),ties_to_image(masks[i]))
            iter+=1
    return imgs.reshape((-1,h,w,3))[motion.flatten()>=.97],iter

def save_motion(out_dir,motion_ties):
    if not (motion_ties is None):
        for i in range(int(np.ceil(float(len(motion_ties))/50.))):
            im = ties_to_image(motion_ties[i*50:min((i+1)*50,len(motion_ties))],5,10)
            cv2.imwrite(out_dir+'/%d_motion.jpg'%(i),im)
        return len(motion_ties)
    else:
        return 0

In [4]:
def generate_ties(dataset,out_dir,length,tie_size,min_r=0,min_roi = 0.5):
    for video_num,(in_dir,out_dir) in enumerate(iterate_folders(dataset,out_dir)):
        ties_num = 0
        motion_ties = None
        for batch_num,(imgs,masks) in enumerate(iterate_bathced(in_dir,length)):
            _,i = process_batch(out_dir,imgs,masks,tie_size,tie_size,min_r,min_roi)
            ties_num += i
            m_ties = get_motion_batch(imgs,masks,tie_size)
            if(len(m_ties) > 0):
                if not(motion_ties is None):
                    motion_ties = np.concatenate((motion_ties,m_ties))
                else:
                    motion_ties = m_ties
        print out_dir+ ' %d ties=%d motion=%d'%(batch_num,ties_num,save_motion(out_dir,motion_ties))

In [5]:
TIE_SIZE = 64
out_prefix = 'data/ties%d'%TIE_SIZE
in_dir = 'data/dataset'
SEQ_LENGTH = 64
generate_ties(in_dir,out_prefix,SEQ_LENGTH,TIE_SIZE,-1.,0.5)

data/ties64/turbulence/turbulence2 61 ties=1240 motion=0
data/ties64/turbulence/turbulence3 20 ties=847 motion=1268
data/ties64/turbulence/turbulence1 42 ties=1694 motion=0
data/ties64/thermal/lakeSide 84 ties=1275 motion=0
data/ties64/thermal/park 4 ties=100 motion=0
data/ties64/thermal/diningRoom 45 ties=690 motion=372
data/ties64/thermal/library 66 ties=1005 motion=18914
data/ties64/PTZ/zoomInZoomOut 8 ties=75 motion=0
data/ties64/PTZ/twoPositionPTZCam 22 ties=240 motion=4
data/ties64/PTZ/intermittentPan 34 ties=720 motion=6
data/ties64/dynamicBackground/overpass 30 ties=434 motion=164
data/ties64/dynamicBackground/fall 45 ties=3542 motion=8992
data/ties64/dynamicBackground/canoe 5 ties=90 motion=0
data/ties64/dynamicBackground/fountain02 14 ties=360 motion=0
data/ties64/dynamicBackground/fountain01 11 ties=288 motion=0
data/ties64/shadow/peopleInShade 13 ties=210 motion=578
data/ties64/shadow/cubicle 97 ties=1470 motion=537
data/ties64/shadow/copyMachine 44 ties=3465 motion=24725
d

In [6]:
TIE_SIZE = 64
out_prefix = 'data/test_ties%d'%TIE_SIZE
in_dir = 'data/test'
SEQ_LENGTH = 64
generate_ties(in_dir,out_prefix,SEQ_LENGTH,TIE_SIZE,-1.,0.5)

data/test_ties64/turbulence/turbulence0 61 ties=2387 motion=0
data/test_ties64/thermal/corridor 75 ties=1140 motion=1866
data/test_ties64/PTZ/continuousPan 16 ties=693 motion=740
data/test_ties64/dynamicBackground/boats 94 ties=475 motion=0
data/test_ties64/shadow/backdoor 24 ties=296 motion=0
data/test_ties64/nightVideos/bridgeEntry 22 ties=131 motion=0
data/test_ties64/intermittentObjectMotion/abandonedBox 31 ties=63 motion=0
data/test_ties64/baseline/highway 18 ties=285 motion=91
data/test_ties64/cameraJitter/badminton 4 ties=385 motion=124
data/test_ties64/lowFramerate/port_0_17fps 30 ties=624 motion=0
data/test_ties64/badWeather/blizzard 94 ties=3696 motion=3184


In [16]:
import shutil
def purge(dir):
    for i in os.listdir(dir):
        for j in os.listdir(dir+'/'+i):
            p = dir+'/'+i+'/'+j
            l = [k[:-4] for k in os.listdir(p)]
            if( not('0_motion' in l)):
                print(p,'no motion ',len(l))
                shutil.rmtree(p)
            else:
                print(p,len(l))
purge('/media/hpc2_storage/nvaulin/ties32')        
purge('/media/hpc2_storage/nvaulin/test_ties32')           

('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence2', 'no motion ', 360)
('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence3', 683)
('/media/hpc2_storage/nvaulin/ties32/turbulence/turbulence1', 661)
('/media/hpc2_storage/nvaulin/ties32/thermal/lakeSide', 139)
('/media/hpc2_storage/nvaulin/ties32/thermal/park', 199)
('/media/hpc2_storage/nvaulin/ties32/thermal/diningRoom', 225)
('/media/hpc2_storage/nvaulin/ties32/thermal/library', 524)
('/media/hpc2_storage/nvaulin/ties32/PTZ/zoomInZoomOut', 'no motion ', 136)
('/media/hpc2_storage/nvaulin/ties32/PTZ/twoPositionPTZCam', 123)
('/media/hpc2_storage/nvaulin/ties32/PTZ/intermittentPan', 384)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/overpass', 136)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/fall', 898)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/canoe', 'no motion ', 102)
('/media/hpc2_storage/nvaulin/ties32/dynamicBackground/fountain02', 'no motion ', 234)
('/media/hpc2_storage/nvaulin/t